In [ ]:
from pathlib import Path
from collections import Counter
import shutil #.copy2
from functools import lru_cache

from tokenizers import ByteLevelBPETokenizer
from tokenizers.normalizers import NFD, Sequence

In [ ]:
with open('utf-refdata/3fb8b25f10e70bc8c4b0246f6ac0ede6.txt', encoding='utf-8') as f:
    print(f.read())

In [ ]:
paths = [str(x) for x in Path(".").glob("utf-refdata/*.txt") if x.stat().st_size > 0]

In [ ]:
%%time 

normalizer = Sequence([NFD()])

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.normalizer = normalizer

# Customize training
tokenizer.train(files=paths, vocab_size=50_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
tokenizer.save("wfctokenizer/tokenizer.json")
tokenizer.save_model('wfctokenizer/')

---

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./wfctokenizer/vocab.json",
    "./wfctokenizer/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("the hindwings are brown with a grey line").tokens

---

### train a language model

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

In [ ]:
from datasets import load_dataset

In [ ]:
with open('ls_refdata', 'r') as f:
    reffiles = [*map(lambda name: 'utf-refdata/'+name[:-1], f.readlines())]
    (reffiles)
        
dataset = load_dataset('text', 
                       data_files=reffiles[:10])

In [ ]:
dataset['train'][8]

In [ ]:
data['train'][0]

---

In [1]:
from transformers import BertTokenizerFast

In [2]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')